In [1]:
import os
import usgeocoder as geo
import pandas as pd
from pyprojroot import here
import requests
from datetime import date
from time import sleep


In [2]:
geo.request_coordinates_geocode((-70.9613897668135, 42.978869))

{'Coordinates': (-70.9613897668135, 42.978869),
 'Date': '2023-08-16',
 'State': 'New Hampshire',
 'County': 'Rockingham',
 'Urban Area': 'Exeter, NH',
 'Census Block': '2041',
 'Census Tract': '610.01'}

In [2]:
os.listdir()

['AddressLatitudeLongitude.csv',
 '.DS_Store',
 'LICENSE',
 'requirements.txt',
 'usgeocoder',
 'tests',
 'test.py',
 'README.md',
 'setup.py',
 '.git',
 'geocoder',
 'test.ipynb']

In [5]:
geocoder = geo.Geocoder()

In [2]:
geo.request_address_geocode('156 Front St, Exeter, NH 03833')

{'Address': '156 Front St, Exeter, NH 03833',
 'Date': '2023-08-13',
 'Latitude': 42.97886968860952,
 'Longitude': -70.96138976681354}

In [5]:
def request_reverse_geocode(latitude, longitude):
    base_geocode_url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates'
    geocode_params = {
        'benchmark': 'Public_AR_Current',
        'format': 'json',
        'vintage': 'Current_Current',
        'y': latitude,
        'x': longitude
    }
    today = date.today().strftime('%Y-%m-%d')
    
    sleep_delay = 0.1
    timeouts = [0.5, 1, 2, 5]
    for t in timeouts:
        try:
            geocode_req = requests.get(base_geocode_url, params=geocode_params, timeout=t)
            geocode_data = geocode_req.json()
            
            # if the request was successful and contains the 'result' key
            if 'result' in geocode_data:
                geographies = geocode_data['result']['geographies']

                return {
                    'Latitude': latitude,
                    'Longitude': longitude,
                    'Date': today,
                    'State': geographies['States'][0]['BASENAME'],
                    'County': geographies['Counties'][0]['BASENAME'],
                    'Urban Area': geographies['Urban Areas'][0]['BASENAME'],
                    'Census Block': geographies['2020 Census Blocks'][0]['BASENAME'],
                    'Census Tract': geographies['Census Tracts'][0]['BASENAME']
                }
            
        except ValueError:
            print(f'Decoding JSON has failed for coordinates: ({latitude}, {longitude})')
            sleep(sleep_delay)
            return None
        
        except requests.exceptions.Timeout:
            if t == timeouts[-1]:
                print(f'All attempts failed for coordinates: ({latitude}, {longitude})')
                sleep(sleep_delay)
                return None
            
            sleep(sleep_delay)
            continue
        
        except requests.exceptions.RequestException as e:
            # Catch any unforeseen requests-related exceptions
            print(f'Request exception occurred for coordinates ({latitude}, {longitude}): {e}')
            sleep(sleep_delay)
            return None

In [25]:
base_geocode_url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates'
geocode_params = {
    'benchmark': 'Public_AR_Current',
    'format': 'json',
    'vintage': 'Current_Current',
    'y': 42.97886968860952,
    'x': None
}
today = date.today().strftime('%Y-%m-%d')

geocode_req = requests.get(base_geocode_url, params=geocode_params, timeout=1)
geocode_data = geocode_req.json()
geocode_data


KeyError: 'result'

In [26]:
geocode_data

{'errors': ['X coordinate cannot be empty'], 'status': '400'}

In [24]:
if 'result' in geocode_data and len(geocode_data['result']['geographies']) == 0:
    print('Not Found')
else:
    print(len(geocode_data['result']['geographies']))
    print(geocode_data['result']['States'])

Not Found


In [27]:
{'Address': '156 Front St, Exeter, NH 03833',
 'Date': '2023-08-13',
 'Latitude': 42.97886968860952,
 'Longitude': -70.96138976681354}

{'Address': '156 Front St, Exeter, NH 03833',
 'Date': '2023-08-13',
 'Latitude': 42.97886968860952,
 'Longitude': -70.96138976681354}

In [7]:
request_reverse_geocode(0, -70.96138976681354)

KeyError: 'States'

In [19]:
base_geocode_url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates'
geocode_params = {
        'benchmark': 'Public_AR_Current',
        'vintage': 'Current_Current',
        'format': 'json',
        'y': 42.97886968860952,
        'x': -70.96138976681354
    }

geocode_req = requests.get(base_geocode_url, params=geocode_params, timeout=5)
geocode_data = geocode_req.json()
geocode_data

{'result': {'geographies': {'States': [{'STATENS': '01779794',
     'GEOID': '33',
     'CENTLAT': '+43.6740329',
     'AREAWATER': 1025957652,
     'STATE': '33',
     'BASENAME': 'New Hampshire',
     'STUSAB': 'NH',
     'OID': '27490227215319',
     'LSADC': '00',
     'FUNCSTAT': 'A',
     'INTPTLAT': '+43.6726907',
     'DIVISION': '1',
     'NAME': 'New Hampshire',
     'REGION': '1',
     'OBJECTID': 8,
     'CENTLON': '-071.5738496',
     'AREALAND': 23190125720,
     'INTPTLON': '-071.5843145',
     'MTFCC': 'G4000'}],
   'County Subdivisions': [{'COUSUB': '25380',
     'GEOID': '3301525380',
     'CENTLAT': '+42.9901527',
     'AREAWATER': 932043,
     'STATE': '33',
     'BASENAME': 'Exeter',
     'OID': '27690308860955',
     'LSADC': '43',
     'FUNCSTAT': 'A',
     'INTPTLAT': '+42.9884525',
     'NAME': 'Exeter town',
     'OBJECTID': 13740,
     'CENTLON': '-070.9646435',
     'COUSUBCC': 'T1',
     'AREALAND': 50772396,
     'INTPTLON': '-070.9615424',
     'MTFCC': '

In [29]:
request_reverse_geocode(-73.961452, 40.714224)

KeyError: 'States'

In [17]:
# get today's date
today = dt.date.today().strftime('%Y-%m-%d')
today

'2023-08-13'

In [8]:
str(here()) + '/test.ipynb'

'/Users/claygendron/Git/Git-Desktop/usgeocoder/test.ipynb'

In [15]:
# convert today to datetime
today = dt.datetime.today()

In [16]:
today

datetime.datetime(2023, 8, 13, 16, 9, 44, 239248)

In [7]:
import pandas as pd
import numpy as np
import requests
from pyprojroot import here
from concurrent.futures import ThreadPoolExecutor

from time import sleep

def request_address_geocode(address):
    sleep_delay = 0.1
    base_geocode_url = 'https://geocoding.geo.census.gov/geocoder/locations/onelineaddress'
    geocode_params = {'benchmark': 'Public_AR_Current', 'format': 'json', 'address': address}
    
    timeouts = [0.5, 1, 2, 5]
    for t in timeouts:
        try:
            geocode_req = requests.get(base_geocode_url, params = geocode_params, timeout = t)
            geocode_data = geocode_req.json()
            
            # If the request was successful but didn't match an address, stop trying and return
            if 'result' in geocode_data and not geocode_data['result']['addressMatches']:
                print(f'Address {address} did not match any records.')
                sleep(sleep_delay)
                return None
            
            # Check if the 'result' key is in the geocode_data dictionary
            if 'result' in geocode_data and geocode_data['result']['addressMatches']:
                coordinates = geocode_data['result']['addressMatches'][0]['coordinates']
                census_latitude = coordinates['y']
                census_longitude = coordinates['x']
                
                sleep(sleep_delay)
                return {'Address': address, 'Latitude': census_latitude, 'Longitude': census_longitude}

        except ValueError:
            print('Decoding JSON has failed for address: ' + address)
            sleep(sleep_delay)
            return None
        
        except requests.exceptions.Timeout:
            if t == timeouts[-1]:
                print(f'All attempts failed for address: {address}')
                sleep(sleep_delay)
            continue
        
        except requests.exceptions.RequestException as e:
            # Catch any unforeseen requests-related exceptions
            print(f'Request exception occurred for address {address}: {e}')
            sleep(sleep_delay)
            return None



In [8]:
address = '156 Front St'
request_address_geocode(address)

Address 156 Front St did not match any records.


In [9]:
addresses_df = pd.read_csv(here('AddressLatitudeLongitude.csv'))

In [14]:
test_100_df = addresses_df.head(10)
test_1000_df = addresses_df.head(1000)

In [15]:
addresses = test_1000_df['Address'].tolist()
addresses = set(addresses)

In [16]:
def batch_geocoder(addresses, n_threads = 100):
    if n_threads > 100:
        # show warning that the max number of threads is 100
        print('WARNING: n_threads is set very high and you may experience rate limits.')
        print('Would you like to set n_threads to the recomended max of 100? (y/n)')
        response = input()
        if response == 'y':
            n_threads = 100
        else:
            print('If this process fails, try reducing n_threads to 100 or less.')
        
        
    addresses = set(addresses)
    result_df = pd.DataFrame(columns = ['Address', 'Latitude', 'Longitude'])
    with ThreadPoolExecutor(max_workers = n_threads) as executor:
        for result in executor.map(request_address_geocode, addresses):
            if result is not None:
                result_df = pd.concat([result_df, pd.DataFrame([result])], ignore_index = True)

    return result_df

In [17]:
result_df = geo.batch_geocoder(addresses)

In [64]:
result_df = pd.DataFrame(columns = ['Address', 'Latitude', 'Longitude'])

# Use ThreadPoolExecutor to run multiple requests in parallel
with ThreadPoolExecutor(max_workers = 100) as executor:
    for result in executor.map(request_address_geocode, addresses):
        if result is not None:
            result_df = pd.concat([result_df, pd.DataFrame([result])], ignore_index = True)

# Save result to CSV
result_df

,Address,Latitude,Longitude
0,"600 Westford St, Lowell, MA 01851",42.633689,-71.337486
1,"40 Countryside Ln, Pelham, NH 03076",42.768863,-71.364502
2,"410 Greenlodge St, Dedham, MA 02026",42.218242,-71.154284
3,"118 Jennison Rd, Milford, NH 03055",42.856435,-71.673721
4,"3 Holly Ct, Exeter, NH 03833",42.977324,-70.933516
5,"11 Dyer Ave, Salem, NH 03079",42.793265,-71.233420
6,"10 Downing St, Laconia, NH 03246",43.534150,-71.483670
7,"114 Ashley Dr, Manchester, NH 03103",42.952850,-71.417385
8,"12698 W 84th Dr, Arvada, CO 80005",39.851634,-105.142778
9,"170 Annand Dr, Milford, NH 03055",42.799873,-71.667777


In [6]:
def get_geocode(address):
    base_geocode_url = 'https://geocoding.geo.census.gov/geocoder/locations/onelineaddress'
    geocode_params = {'benchmark': 'Public_AR_Current', 'format': 'json', 'address': address}
    geocode_req = requests.get(base_geocode_url, params = geocode_params)

    try:
        geocode_data = geocode_req.json()
    except ValueError:  # includes simplejson.decoder.JSONDecodeError
        print('Decoding JSON has failed for address: ' + address)
        return None

    # Check if the 'result' key is in the geocode_data dictionary
    if 'result' in geocode_data and geocode_data['result']['addressMatches']:
        # Get coordinates from Geocoding API response
        coordinates = geocode_data['result']['addressMatches'][0]['coordinates']

        # Request to FCC Census Block API
        census_params = {'latitude': coordinates['y'], 'longitude': coordinates['x'], 'format': 'json'}

        # Get Census Tract from FCC API response
        census_latitude = census_params['latitude']
        census_longitude = census_params['longitude']

        return {'Address': address, 'Latitude': census_latitude, 'Longitude': census_longitude}
    else:
        return None
    

In [7]:
address = '156 Front St, Exeter, NH 03833'
get_geocode(address)

KeyboardInterrupt: 

In [1]:
import requests

# Create a session
session = requests.Session()

def get_geocode(address):
    base_geocode_url = 'https://geocoding.geo.census.gov/geocoder/locations/onelineaddress'
    geocode_params = {'benchmark': 'Public_AR_Current', 'format': 'json', 'address': address}
    
    # Use the session object to make the request
    geocode_req = session.get(base_geocode_url, params=geocode_params)

    try:
        geocode_data = geocode_req.json()
    except ValueError:  # includes simplejson.decoder.JSONDecodeError
        print('Decoding JSON has failed for address: ' + address)
        return None

    # Check if the 'result' key is in the geocode_data dictionary
    if 'result' in geocode_data and geocode_data['result']['addressMatches']:
        # Get coordinates from Geocoding API response
        coordinates = geocode_data['result']['addressMatches'][0]['coordinates']

        # Request to FCC Census Block API
        census_params = {'latitude': coordinates['y'], 'longitude': coordinates['x'], 'format': 'json'}

        # Get Census Tract from FCC API response
        census_latitude = census_params['latitude']
        census_longitude = census_params['longitude']

        return {'Address': address, 'Latitude': census_latitude, 'Longitude': census_longitude}
    else:
        return None

address = '156 Front St, Exeter, NH 03833'
get_geocode(address)


{'Address': '156 Front St, Exeter, NH 03833',
 'Latitude': 42.97886968860952,
 'Longitude': -70.96138976681354}

In [176]:
# create a new column of the full address
def apply_concatenate_address(row):
    try:
        address_parts = [
            str(row['Address 1']).strip() if pd.notna(row['Address 1']) else '',
            ' ' + str(row['Address 2']).strip() if pd.notna(row['Address 2']) else '',
            ', ' + str(row['City']).strip() if pd.notna(row['City']) else '',
            ', ' + str(row['State']).strip() if pd.notna(row['State']) else '',
            ' ' + str(row['ZIP']).strip()[0:5] if pd.notna(row['ZIP']) and len(str(row['ZIP']).strip()) >= 5  else ''
        ]
        # remove any empty strings and join all parts
        address_parts = [part for part in address_parts if part.strip()]
        address = ''.join(address_parts)
        
        return address
    
    except Exception as e:
        # Log the error and the row for debugging purposes
        print(f'Error processing row: {row}. Error: {e}')
        print('The dataframe must have the following columns: Address 1, Address 2, City, State, ZIP')
        return None
    
    
def create_address_list(df):
    required_cols = {'Address 1', 'Address 2', 'City', 'State', 'ZIP'}
    if len(df.columns) == 5:
        df.columns = required_cols
    elif not required_cols.issubset(set(df.columns)):
        raise Exception('The dataframe must have the following columns: Address 1, Address 2, City, State, ZIP')
    
    addresses = df.apply(apply_concatenate_address, axis = 1)
    addresses_list = list(set(addresses)) # remove any duplicate addresses and convert to list
    addresses_list = [address for address in addresses_list if address.strip()] # remove any empty strings
    
    if len(addresses_list) == 0:
        raise Exception('No addresses were found in the dataframe. Please check the column names and try again.')
    
    return addresses_list


In [179]:
def create_address_list(df):
    required_cols = {'Address 1', 'Address 2', 'City', 'State', 'ZIP'}
    if len(df.columns) == 5:
        df.columns = required_cols
    elif not required_cols.issubset(set(df.columns)):
        raise Exception('The dataframe must have the following columns: Address 1, Address 2, City, State, ZIP')
    
    addresses = df.apply(apply_concatenate_address, axis = 1)
    addresses_list = list(set(addresses)) # remove any duplicate addresses and convert to list
    addresses_list = [address for address in addresses_list if address.strip()] # remove any empty strings
    
    if len(addresses_list) == 0:
        raise Exception('No addresses were found in the dataframe. Please check the column names and try again.')
    
    return addresses_list

In [180]:
test_1000_df

,Address,Latitude,Longitude
0,"114 Ashley Dr, Manchester, NH 03103",42.952850,-71.417385
1,"600 Westford St, Lowell, MA 01851",42.633689,-71.337486
2,"118 Jennison Rd, Milford, NH 03055",42.856435,-71.673721
3,"3 Holly Ct, Exeter, NH 03833",42.977324,-70.933516
4,"10 Downing St, Laconia, NH 03246",43.534150,-71.483670
...,...,...,...
4995,"104 Edgewater Rd, Mashpee, MA 02649",41.656960,-70.458504
4996,"66 Robin Hood Rd, Nashua, NH 03062",42.731290,-71.482837
4997,"410 Wareham St, Middleboro, MA 02346",41.851815,-70.854377
4998,"88b Bowers St, Nashua, NH 03060",42.756808,-71.452243


In [178]:
df = pd.DataFrame()

create_address_list(df)

Exception: The dataframe must have 5 columns: Address 1, Address 2, City, State, ZIP

In [173]:
address = {
    'Address 1': '   123 Main St',
    'Address 2': ' Test     ',
    
    'City': '  New York    ',
    'State': '   NY   ',
    'ZIP': ' 10    '
}

In [174]:
address

{'Address 1': '   123 Main St',
 'Address 2': ' Test     ',
 'City': '  New York    ',
 'State': '   NY   ',
 'ZIP': ' 10    '}

In [175]:
concatenate_address(address)

'123 Main St Test, New York, NY'